In [3]:

# import required packages
import cv2
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

In [4]:

# Initialize image data generator with rescaling
train_data_gen = ImageDataGenerator(rescale=1./255)
validation_data_gen = ImageDataGenerator(rescale=1./255)

# Preprocess all test images
train_generator = train_data_gen.flow_from_directory(
        r'C:\Users\adi\Desktop\face detection based attendance system\divided\train',
        target_size=(48, 48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')

# Preprocess all train images
validation_generator = validation_data_gen.flow_from_directory(
        r'C:\Users\adi\Desktop\face detection based attendance system\divided\val',
        target_size=(48, 48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')

Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.


In [5]:
list=validation_generator.class_indices
lable=dict([(value,key)for key,value in list.items()])
print(lable)

{}


In [8]:
# create model structure
model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4, activation='softmax'))

cv2.ocl.setUseOpenCL(False)

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(learning_rate=0.0001, decay=1e-6), 
              metrics=['accuracy'])

In [10]:
# Train the neural network/model
emotion_model_info = model.fit(
        train_generator,
        steps_per_epoch=35,
        epochs=5,
        validation_data=validation_generator,
        )

Epoch 1/5
35/35 [==============================] - 11s 309ms/step - loss: 0.0425 - accuracy: 0.9897 - val_loss: 0.0156 - val_accuracy: 0.9963
Epoch 2/5
35/35 [==============================] - 10s 296ms/step - loss: 0.0414 - accuracy: 0.9875 - val_loss: 0.0152 - val_accuracy: 0.9963
Epoch 3/5
35/35 [==============================] - 10s 296ms/step - loss: 0.0302 - accuracy: 0.9888 - val_loss: 0.0146 - val_accuracy: 0.9956
Epoch 4/5
35/35 [==============================] - 10s 294ms/step - loss: 0.0387 - accuracy: 0.9902 - val_loss: 0.0074 - val_accuracy: 0.9988
Epoch 5/5
35/35 [==============================] - 11s 317ms/step - loss: 0.0280 - accuracy: 0.9906 - val_loss: 0.0066 - val_accuracy: 0.9988


In [11]:
# save model structure in jason file
model_json = emotion_model.to_json()
with open("e_model.json", "w") as json_file:
    json_file.write(model_json)

# save trained model weight in .h5 file
emotion_model.save_weights('e_model.h5')

NameError: name 'emotion_model' is not defined

In [2]:
import cv2
import numpy as np
from keras.models import model_from_json
from datetime import datetime
import os
import csv

# load json and create model
json_file = open('e_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
emotion_model = model_from_json(loaded_model_json)

# load weights into new model
emotion_model.load_weights("e_model.h5")
print("Loaded model from disk")

# start the webcam feed
cap = cv2.VideoCapture(0)

# pass here your video path
# you may download one from here : https://www.pexels.com/video/three-girls-laughing-5273028/
#cap = cv2.VideoCapture("C:\\JustDoIt\\ML\\Sample_videos\\emotion_sample6.mp4")

while True:
    # Find haar cascade to draw bounding box around face
    ret, frame = cap.read()
    frame = cv2.resize(frame, (1280, 720))
    if not ret:
        break
    face_detector = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # detect faces available on camera
    num_faces = face_detector.detectMultiScale(gray_frame, scaleFactor=1.3, minNeighbors=5)

    # take each face available on the camera and Preprocess it
    for (x, y, w, h) in num_faces:
        cv2.rectangle(frame, (x, y-50), (x+w, y+h+10), (0, 255, 0), 4)
        roi_gray_frame = gray_frame[y:y + h, x:x + w]
        cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray_frame, (48, 48)), -1), 0)

        # predict the emotions
        emotion_prediction = emotion_model.predict(cropped_img)
        maxindex = int(np.argmax(emotion_prediction))
        cv2.putText(frame, lable[maxindex], (x+5, y-20), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
        
 
        name=lable[maxindex]
        with open("attend.csv", "r+")as f:
            mydata=f.readlines()
            namelist=[]
            print(mydata)
            for line in mydata:
                entry=line.split(',')
                namelist.append(entry[0])
            today = datetime.now()
            time=today.strftime("%H:%M:%S")
            date=today.strftime("%d/%m/%y")
            check=name+" "+date+"\n"
            print(check)
            if check not in mydata:
                f.writelines(f'{name} {date}\n')

    cv2.imshow('Emotion Detection', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Loaded model from disk


NameError: name 'lable' is not defined